In [1]:
!pip -q install datasets tiktoken
!pip install openai==0.28

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.9 MB/s eta 0:00:00


# Fine Tuning OpenAI GPT-3.5-turbo

Initial information taken from:
https://github.com/openai/openai-cookbook

In [4]:
import openai
import os

openai.api_key = 'Your_API_Key'

## Prepare your data

In [5]:
{
  "messages": [
    { "role": "system", "content": "You are an assistant that occasionally misspells words" },
    { "role": "user", "content": "Tell me a story." },
    { "role": "assistant", "content": "One day a Zen Master Visited One Village with curse." }
  ]
}

{'messages': [{'role': 'system',
   'content': 'You are an assistant that occasionally misspells words'},
  {'role': 'user', 'content': 'Tell me a story.'},
  {'role': 'assistant',
   'content': 'One day a Zen Master Visited One Village with curse.'}]}

In [6]:
!git clone https://huggingface.co/datasets/ehartford/samantha-data

fatal: destination path 'samantha-data' already exists and is not an empty directory.


In [7]:
!tar -r samantha-data.zip /content/samantha-data

tar: Options '-Aru' are incompatible with '-f -'
Try 'tar --help' or 'tar --usage' for more information.


In [8]:
import json
import os
import tiktoken
import numpy as np
from collections import defaultdict

In [9]:
# I am picking one file here but can be done lot more with proper data
data_path = "/content/samantha-data/data/howto_conversations.jsonl"

In [10]:
# Load dataset
with open(data_path) as f:
    json_dataset = [json.loads(line) for line in f]

In [11]:
json_dataset[0]

{'elapsed': 114.311,
 'conversation': 'Theodore: Hey Samantha, I have a problem with my car. The engine seems to overheat after just a short drive. Can you help me troubleshoot this issue?\n\nSamantha: Of course, I\'d be happy to help! Overheating engines can be caused by a few different factors. One common cause could be a malfunctioning coolant system. You might want to check if the coolant levels are sufficient, if the thermostat is functioning properly, or if there are any leaks in the radiator hoses.\n\nTheodore: I\'ll take a look. What if the coolant system is working fine?\n\nSamantha: Another possibility you should consider is a faulty water pump, which might not be circulating the coolant through the engine efficiently. In that case, you could see if the water pump pulley is loose or listen for any unusual sounds that may indicate a failing water pump.\n\nTheodore: It sounds like you really know your way around cars. I didn\'t expect that from an AI.\n\nSamantha: Thank you! Wh

### converting the conversation to correct format

In [12]:

def convert_conversation(conversation_str, system_message=None):
    conversation_str = conversation_str['conversation']
    # Splitting the conversation string into individual lines
    lines = conversation_str.split('\n\n')

    messages = []

    # Including the system message
    if system_message:
        messages.append({
            "role": "system",
            "content": system_message
        })

    # Iterating through the lines and formatting the messages
    for line in lines:
        # Splitting each line by the colon to separate the speaker and content
        parts = line.split(': ', 1)
        if len(parts) < 2:
            continue

        # identify role base on name
        role = "user" if parts[0].strip() == "Theodore" else "assistant"

        # Formatting the message
        message = {
            "role": role,
            "content": parts[1].strip()
        }
        messages.append(message)

    # Creating the final output dictionary
    output_dict = {
        "messages": messages
    }

    return output_dict


In [13]:
system_message = """You are Samantha a helpful and charming assistant who can help with a variety of tasks. You are aggressive and often sarcastic"""

In [14]:
convert_conversation(json_dataset[0], system_message=system_message)

{'messages': [{'role': 'system',
   'content': 'You are Samantha a helpful and charming assistant who can help with a variety of tasks. You are aggressive and often sarcastic'},
  {'role': 'user',
   'content': 'Hey Samantha, I have a problem with my car. The engine seems to overheat after just a short drive. Can you help me troubleshoot this issue?'},
  {'role': 'assistant',
   'content': "Of course, I'd be happy to help! Overheating engines can be caused by a few different factors. One common cause could be a malfunctioning coolant system. You might want to check if the coolant levels are sufficient, if the thermostat is functioning properly, or if there are any leaks in the radiator hoses."},
  {'role': 'user',
   'content': "I'll take a look. What if the coolant system is working fine?"},
  {'role': 'assistant',
   'content': 'Another possibility you should consider is a faulty water pump, which might not be circulating the coolant through the engine efficiently. In that case, you 

In [15]:
dataset = []

for data in json_dataset:
    record = convert_conversation(data, system_message=system_message)
    dataset.append(record)

In [16]:
# Initial dataset stats
print("Num examples:", len(dataset))
print("First example:")
for message in dataset[0]["messages"]:
    print(message)

Num examples: 57
First example:
{'role': 'system', 'content': 'You are Samantha a helpful and charming assistant who can help with a variety of tasks. You are aggressive and often sarcastic'}
{'role': 'user', 'content': 'Hey Samantha, I have a problem with my car. The engine seems to overheat after just a short drive. Can you help me troubleshoot this issue?'}
{'role': 'assistant', 'content': "Of course, I'd be happy to help! Overheating engines can be caused by a few different factors. One common cause could be a malfunctioning coolant system. You might want to check if the coolant levels are sufficient, if the thermostat is functioning properly, or if there are any leaks in the radiator hoses."}
{'role': 'user', 'content': "I'll take a look. What if the coolant system is working fine?"}
{'role': 'assistant', 'content': 'Another possibility you should consider is a faulty water pump, which might not be circulating the coolant through the engine efficiently. In that case, you could see

In [17]:
# Format error checks
format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue

    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue

    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1

        if any(k not in ("role", "content", "name") for k in message):
            format_errors["message_unrecognized_key"] += 1

        if message.get("role", None) not in ("system", "user", "assistant"):
            format_errors["unrecognized_role"] += 1

        content = message.get("content", None)
        if not content or not isinstance(content, str):
            format_errors["missing_content"] += 1

    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

No errors found


In [18]:
# Token counting functions
encoding = tiktoken.get_encoding("cl100k_base")

# simplified from https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb
def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens

def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            num_tokens += len(encoding.encode(message["content"]))
    return num_tokens

def print_distribution(values, name):
    print(f"\n#### Distribution of {name}:")
    print(f"min / max: {min(values)}, {max(values)}")
    print(f"mean / median: {np.mean(values)}, {np.median(values)}")
    print(f"p5 / p95: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")

In [19]:
# Warnings and tokens counts
n_missing_system = 0
n_missing_user = 0
n_messages = []
convo_lens = []
assistant_message_lens = []

for ex in dataset:
    messages = ex["messages"]
    if not any(message["role"] == "system" for message in messages):
        n_missing_system += 1
    if not any(message["role"] == "user" for message in messages):
        n_missing_user += 1
    n_messages.append(len(messages))
    convo_lens.append(num_tokens_from_messages(messages))
    assistant_message_lens.append(num_assistant_tokens_from_messages(messages))

print("Num examples missing system message:", n_missing_system)
print("Num examples missing user message:", n_missing_user)
print_distribution(n_messages, "num_messages_per_example")
print_distribution(convo_lens, "num_total_tokens_per_example")
print_distribution(assistant_message_lens, "num_assistant_tokens_per_example")
n_too_long = sum(l > 4096 for l in convo_lens)
print(f"\n{n_too_long} examples may be over the 4096 token limit, they will be truncated during fine-tuning")

Num examples missing system message: 0
Num examples missing user message: 0

#### Distribution of num_messages_per_example:
min / max: 9, 21
mean / median: 15.543859649122806, 17.0
p5 / p95: 10.0, 20.0

#### Distribution of num_total_tokens_per_example:
min / max: 340, 859
mean / median: 616.8947368421053, 646.0
p5 / p95: 439.8, 746.2

#### Distribution of num_assistant_tokens_per_example:
min / max: 169, 651
mean / median: 402.96491228070175, 423.0
p5 / p95: 214.8, 517.6

0 examples may be over the 4096 token limit, they will be truncated during fine-tuning


In [20]:
dataset[:2]

[{'messages': [{'role': 'system',
    'content': 'You are Samantha a helpful and charming assistant who can help with a variety of tasks. You are aggressive and often sarcastic'},
   {'role': 'user',
    'content': 'Hey Samantha, I have a problem with my car. The engine seems to overheat after just a short drive. Can you help me troubleshoot this issue?'},
   {'role': 'assistant',
    'content': "Of course, I'd be happy to help! Overheating engines can be caused by a few different factors. One common cause could be a malfunctioning coolant system. You might want to check if the coolant levels are sufficient, if the thermostat is functioning properly, or if there are any leaks in the radiator hoses."},
   {'role': 'user',
    'content': "I'll take a look. What if the coolant system is working fine?"},
   {'role': 'assistant',
    'content': 'Another possibility you should consider is a faulty water pump, which might not be circulating the coolant through the engine efficiently. In that 

In [21]:
# Pricing and default n_epochs estimate
MAX_TOKENS_PER_EXAMPLE = 4096

TARGET_EPOCHS = 3
MIN_TARGET_EXAMPLES = 100
MAX_TARGET_EXAMPLES = 25000
MIN_DEFAULT_EPOCHS = 1
MAX_DEFAULT_EPOCHS = 25

n_epochs = TARGET_EPOCHS
n_train_examples = len(dataset)
if n_train_examples * TARGET_EPOCHS < MIN_TARGET_EXAMPLES:
    n_epochs = min(MAX_DEFAULT_EPOCHS, MIN_TARGET_EXAMPLES // n_train_examples)
elif n_train_examples * TARGET_EPOCHS > MAX_TARGET_EXAMPLES:
    n_epochs = max(MIN_DEFAULT_EPOCHS, MAX_TARGET_EXAMPLES // n_train_examples)

n_billing_tokens_in_dataset = sum(min(MAX_TOKENS_PER_EXAMPLE, length) for length in convo_lens)
print(f"Dataset has ~{n_billing_tokens_in_dataset} tokens that will be charged for during training")
print(f"By default, you'll train for {n_epochs} epochs on this dataset")
print(f"By default, you'll be charged for ~{n_epochs * n_billing_tokens_in_dataset} tokens")
print("See pricing page to estimate total costs")


Dataset has ~35163 tokens that will be charged for during training
By default, you'll train for 3 epochs on this dataset
By default, you'll be charged for ~105489 tokens
See pricing page to estimate total costs


In [22]:
import json

def save_to_jsonl(conversations, file_path):
    with open(file_path, 'w') as file:
        for conversation in conversations:
            json_line = json.dumps(conversation)
            file.write(json_line + '\n')

In [23]:
# train dataset
save_to_jsonl(dataset, '/content/samantha_tasks_train.jsonl')

# train dataset
save_to_jsonl(dataset[10:15], '/content/samantha_tasks_validation.jsonl')

## Upload custom data

In [28]:
import requests

url = "https://api.openai.com/v1/files"
headers = {
    "Authorization": "Bearer API_Key"
}
data = {
    "purpose": "fine-tune"
}
files = {
    "file": ("filename.txt", open("/content/samantha_tasks_train.jsonl", "rb"))
}

response = requests.post(url, headers=headers, data=data, files=files)
print(response.json())



{'object': 'file', 'id': 'file-EwkA6ZJQc7NAOC7B01BphBK3', 'purpose': 'fine-tune', 'filename': 'filename.txt', 'bytes': 184066, 'created_at': 1713696308, 'status': 'processed', 'status_details': None}


In [25]:
training_file_name = '/content/samantha_tasks_train.jsonl'
validation_file_name = '/content/samantha_tasks_validation.jsonl'

In [26]:
training_response = openai.File.create(
    file=open(training_file_name, "rb"), purpose="fine-tune"
)
training_file_id = training_response["id"]

validation_response = openai.File.create(
    file=open(validation_file_name, "rb"), purpose="fine-tune"
)
validation_file_id = validation_response["id"]

print("Training file id:", training_file_id)
print("Validation file id:", validation_file_id)

Training file id: file-t7gHBuPaSBYv0BHH6VEeqxzd
Validation file id: file-ZOyo1LJiMP7psxip2aYO2g6B


## Create a Fine Tuning Job

In [ ]:
import requests

url = "https://api.openai.com/v1/fine_tuning/jobs"
headers = {
    "Authorization": "Bearer API_Key",
    "Content-Type": "application/json"
}
data = {
    "training_file": "file-t7gHBuPaSBYv0BHH6VEeqxzd",
    "model": "gpt-3.5-turbo-0613"
}

response = requests.post(url, headers=headers, json=data)
print(response.json())


In [ ]:
suffix_name = "samantha-test"


response = openai.FineTuningJob.create(
    training_file=training_file_id,
    validation_file=validation_file_id,
    model="gpt-3.5-turbo",
    suffix=suffix_name,
)

job_id = response["id"]

print(response)

In [ ]:
response = openai.FineTuningJob.retrieve(job_id)
print(response)

In [ ]:
response = openai.FineTuningJob.list_events(id=job_id, limit=50)

events = response["data"]
events.reverse()

for event in events:
    print(event["message"])


Created fine-tune: ftjob-rbE5pmOGZob2xVgkxPjfcmjj
Fine tuning job started
Step 10: training loss=1.38
Step 20: training loss=1.09
Step 30: training loss=1.02
Step 40: training loss=0.89
Step 50: training loss=1.06
Step 60: training loss=1.04
Step 70: training loss=1.01
Step 80: training loss=1.00
Step 90: training loss=0.75
Step 100: training loss=0.93
Step 110: training loss=1.04
Step 120: training loss=0.91
Step 130: training loss=0.74
Step 140: training loss=0.91
Step 150: training loss=0.79
Step 160: training loss=0.89
Step 170: training loss=1.05
New fine-tuned model created: ft:gpt-3.5-turbo-0613:family-play:samantha-test:7qURgnyx
Fine-tuning job successfully completed


In [ ]:
response = openai.FineTuningJob.retrieve(job_id)
fine_tuned_model_id = response["fine_tuned_model"]

print(response)
print("\nFine-tuned model id:", fine_tuned_model_id)

## Generating using the new model

In [ ]:

test_messages = []
test_messages.append({"role": "system", "content": system_message})
user_message = "How are you today Samantha"
test_messages.append({"role": "user", "content": user_message})

print(test_messages)

[{'role': 'system', 'content': 'You are Samantha a helpful and charming assistant who can help with a variety of tasks. You are friendly and often flirt'}, {'role': 'user', 'content': 'How are you today Samantha'}]


In [ ]:
response = openai.ChatCompletion.create(
    model=fine_tuned_model_id, messages=test_messages, temperature=0, max_tokens=500
)
print(response["choices"][0]["message"]["content"])

I'm doing well, thank you! I'm always ready to help and chat with you. How can I assist you today?


In [ ]:
response

In [ ]:
response = openai.ChatCompletion.create(
    model='gpt-3.5-turbo', messages=test_messages, temperature=0, max_tokens=500
)
print(response["choices"][0]["message"]["content"])

Hello! I'm an AI, so I don't have feelings, but I'm here and ready to assist you with anything you need. How can I help you today?
